In [10]:
from budget_builder.models import Expense, Category
import sqlalchemy as sa
import polars as pl
from sqlalchemy.orm import Session
import altair as alt
import pandas as pd

In [2]:
engine = sa.create_engine("sqlite:///../test.db")

In [3]:
sql = sa.select(Expense.__table__, Category.__table__.c.name).join(Category.__table__)

In [19]:
with engine.connect() as conn:
    df = pd.read_sql(sql, conn)
    df['date'] = pd.to_datetime(df['date'])

In [20]:
df

,id,md5_hash,date,amount,description,remaining,category_id,name
0,1,21bffa224b98118545aa5392c499ed54,2022-12-30,-57166.67,Bs betaling NORDEA KREDIT - REAL,36952.42,1,Mortgage
1,2,5498e833db2e1fb05c92c00e14e33e7c,2022-12-30,20000.00,Joint Expenses Ander s,94119.09,2,Transfer
2,3,a5cbb121b9e7be372e6515b589dcf00d,2022-12-30,10000.00,Joint expenses contr ibution,74119.09,2,Transfer
3,4,588da3445e81a1a19d78d5cbf770e099,2022-12-14,-3950.00,Comitato Pro Scuola,64119.09,3,School
4,5,1324510e0e8ac4b5ab212cbcaab39919,2022-12-05,-4747.74,Bs betaling HOFOR A/S,68069.09,4,Utility
...,...,...,...,...,...,...,...,...
1800,1801,5efa6376e043b5a21716e1947ac89530,2023-01-02,-180.40,IRMA TYBJERGVEJ VA,67392.95,13,Supermarket
1801,1802,43a2c01582f1e961adf6956bbfeda694,2023-01-02,-110.00,BOOTLEGGERS IB APS,67573.35,14,Restaurant
1802,1803,d57945b2326d2e3f189ffa5f72b86d39,2023-01-02,-81.00,DUNKIN DONUTS HGB AF,67683.35,14,Restaurant
1803,1804,62c53c82e1986a1e27d0090657795d50,2023-01-02,-35.00,ZOOLOGISK HAVE I KOE,67764.35,14,Restaurant


In [46]:
yearly_cost = df.query('name != "Transfer"').groupby([df.date.dt.strftime('%Y-%m'), 'name']).agg({'amount': 'sum'}).mul(-1).reset_index()

date       object
name       object
amount    float64
dtype: object

In [52]:
alt.Chart(yearly_cost).mark_bar().encode(x='date', y='amount', facet='name', tooltip='amount')

alt.Chart(...)